In [ ]:
pip install texthero

In [4]:
import sagemaker, boto3, json
from sagemaker.session import Session

In [5]:
sagemaker_session = Session()
aws_role = sagemaker_session.get_caller_identity_arn()
aws_region = boto3.Session().region_name
sess = sagemaker.Session()

In [6]:
#path of first input test file
path='./sampledocs/txt/'

In [ ]:
# let us scan the full directory, read the text files and clean them using texthero
import pandas as pd
import texthero as hero
docName=[]
docType=[]
docText=[]
import glob
list_of_files = glob.glob(path+'*.txt')           # create the list of file
fileNames=[]
for file_name in list_of_files:
    f = open(file_name,'r')
    fileText=f.read()
    docName.append(file_name)
    docType.append('txt')
    docText.append(fileText)
fullDocs = pd.DataFrame({'Name':docName,'Type':docType,'Text':docText})
fullDocs['cleanText']=hero.clean(fullDocs['Text'])

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')


/opt/conda/lib/python3.7/site-packages/pkg_resources/__init__.py:119: PkgResourcesDeprecationWarning: 4.0.0-unsupported is an invalid version and will not be supported in a future release
  PkgResourcesDeprecationWarning,


In [8]:
fullDocs['text_word_count'] = fullDocs['Text'].apply(lambda x: len(x.strip().split()))  # word count
fullDocs['text_unique_words']=fullDocs['Text'].apply(lambda x:len(set(str(x).split())))  # number of unique words
fullDocs.head()

,Name,Type,Text,cleanText,text_word_count,text_unique_words
0,./sampledocs/txt/Technology-and-innovation-in-...,txt,Technology and\ninnovation in the\ninsurance s...,technology innovation insurance sector technol...,16742,4228
1,./sampledocs/txt/ai-360-research.txt,txt,AI 360: insights from the\nnext frontier of bu...,ai insights next frontier business corner offi...,5281,1746
2,./sampledocs/txt/Module-1-Lecture-Slides.txt,txt,"Application of AI, Insurtech and Real Estate\n...",application ai insurtech real estate technolog...,3728,1506
3,./sampledocs/txt/Insurance-2030-The-impact-of-...,txt,Insurance Practice\n\nInsurance 2030—\nThe imp...,insurance practice insurance -- impact ai futu...,4424,1782
4,./sampledocs/txt/sigma-5-2020-en.txt,txt,No 5 /2020\n\nMachine intelligence in\ninsuran...,machine intelligence insurance insights end en...,14478,4329


In [25]:
# Run this cell to import the Data Wrangler widget to show automatic visualization and generate code to fix data quality issues

import sagemaker_datawrangler
display(fullDocs)

# Display Pandas DataFrame to view the widget: df, display(df), df.sample()... 

                                                 Name Type  \
0   ./sampledocs/txt/Technology-and-innovation-in-...  txt   
1                ./sampledocs/txt/ai-360-research.txt  txt   
2        ./sampledocs/txt/Module-1-Lecture-Slides.txt  txt   
3   ./sampledocs/txt/Insurance-2030-The-impact-of-...  txt   
4                ./sampledocs/txt/sigma-5-2020-en.txt  txt   
5   ./sampledocs/txt/Issues_Paper_on_Increasing_Di...  txt   
6   ./sampledocs/txt/Data machine the insurers usi...  txt   
7   ./sampledocs/txt/Artificial Financial Intellig...  txt   
8          ./sampledocs/txt/ai_in_insurance_web_0.txt  txt   
9                 ./sampledocs/txt/sigma1_2020_en.txt  txt   
10  ./sampledocs/txt/Impact-Big-Data-AI-in-the-Ins...  txt   
11                  ./sampledocs/txt/ai-insurance.txt  txt   
12  ./sampledocs/txt/Digital-disruption-in-Insuran...  txt   
13  ./sampledocs/txt/Innovation_Artificial-Intelli...  txt   
14  ./sampledocs/txt/WEF_Governance_of_Chatbots_in...  txt   
15  ./sa

In [10]:


model_id, model_version, = (
    "huggingface-text2text-flan-t5-xl",
    "*",
)



In [11]:


from ipywidgets import Dropdown
from sagemaker.jumpstart.notebook_utils import list_jumpstart_models

# Retrieves all Text Generation models available by SageMaker Built-In Algorithms.
filter_value = "task == text2text"
text_generation_models = list_jumpstart_models(filter=filter_value)

# display the model-ids in a dropdown to select a model for inference.
model_dropdown = Dropdown(
    options=text_generation_models,
    value=model_id,
    description="Select a model",
    style={"description_width": "initial"},
    layout={"width": "max-content"},
)



In [12]:


display(model_dropdown)



Dropdown(description='Select a model', index=8, layout=Layout(width='max-content'), options=('huggingface-text…

In [13]:


# model_version="*" fetches the latest version of the model
model_id, model_version = model_dropdown.value, "*"



In [14]:


def get_sagemaker_session(local_download_dir) -> sagemaker.Session:
    """Return the SageMaker session."""

    sagemaker_client = boto3.client(
        service_name="sagemaker", region_name=boto3.Session().region_name
    )

    session_settings = sagemaker.session_settings.SessionSettings(
        local_download_dir=local_download_dir
    )

    # the unit test will ensure you do not commit this change
    session = sagemaker.session.Session(
        sagemaker_client=sagemaker_client, settings=session_settings
    )

    return session



In [15]:
!mkdir -p download_dir

In [16]:
from sagemaker import image_uris, model_uris, script_uris, hyperparameters
from sagemaker.model import Model
from sagemaker.predictor import Predictor
from sagemaker.utils import name_from_base


endpoint_name = name_from_base(f"jumpstart-example-{model_id}")

inference_instance_type = "ml.p3.2xlarge"

# Retrieve the inference docker container uri. This is the base HuggingFace container image for the default model above.
deploy_image_uri = image_uris.retrieve(
    region=None,
    framework=None,  # automatically inferred from model_id
    image_scope="inference",
    model_id=model_id,
    model_version=model_version,
    instance_type=inference_instance_type,
)

# Retrieve the inference script uri. This includes all dependencies and scripts for model loading, inference handling etc.
deploy_source_uri = script_uris.retrieve(
    model_id=model_id, model_version=model_version, script_scope="inference"
)


# Retrieve the model uri.
model_uri = model_uris.retrieve(
    model_id=model_id, model_version=model_version, model_scope="inference"
)

if model_id in [
    "huggingface-text2text-flan-t5-xl",
    "huggingface-text2text-flan-t5-large",
]:  # For those large models,
    # we already repack the inference script and model artifacts for you
    # Create the SageMaker model instance
    model = Model(
        image_uri=deploy_image_uri,
        model_data=model_uri,
        role=aws_role,
        predictor_cls=Predictor,
        name=endpoint_name,
    )
else:
    # Create the SageMaker model instance
    model = Model(
        image_uri=deploy_image_uri,
        source_dir=deploy_source_uri,
        model_data=model_uri,
        entry_point="inference.py",  # entry point file in source_dir and present in deploy_source_uri
        role=aws_role,
        predictor_cls=Predictor,
        name=endpoint_name,
        sagemaker_session=get_sagemaker_session("download_dir"),
    )

# deploy the Model. Note that we need to pass Predictor class when we deploy model through Model class,
# for being able to run inference through the sagemaker API.
model_predictor = model.deploy(
    initial_instance_count=1,
    instance_type=inference_instance_type,
    predictor_cls=Predictor,
    endpoint_name=endpoint_name,
    volume_size=30,
)


------------!

In [17]:
newline, bold, unbold = "\n", "\033[1m", "\033[0m"


def query_endpoint(encoded_text, endpoint_name):
    client = boto3.client("runtime.sagemaker")
    response = client.invoke_endpoint(
        EndpointName=endpoint_name, ContentType="application/x-text", Body=encoded_text
    )
    return response


def parse_response(query_response):
    model_predictions = json.loads(query_response["Body"].read())
    generated_text = model_predictions["generated_text"]
    return generated_text

In [18]:


newline, bold, unbold = "\n", "\033[1m", "\033[0m"

text1 = "Translate to German:  My name is Arthur"
text2 = "A step by step recipe to make bolognese pasta:"


for text in [text1, text2]:
    query_response = query_endpoint(text.encode("utf-8"), endpoint_name=endpoint_name)
    generated_text = parse_response(query_response)
    print(
        f"Inference:{newline}"
        f"input text: {text}{newline}"
        f"generated text: {bold}{generated_text}{unbold}{newline}"
    )



Inference:
input text: Translate to German:  My name is Arthur
generated text: Ich bin Arthur.

Inference:
input text: A step by step recipe to make bolognese pasta:
generated text: In a large saucepan, combine the ground beef, onion, garlic, tomato paste, tomato



In [19]:
# Input must be a json
payload = {
    "text_inputs": "A step by step recipe to make cheese fondue:",
    "max_length": 50,
    "num_return_sequences": 5,
    "top_k": 50,
    "top_p": 0.95,
    "do_sample": True,
}


def query_endpoint_with_json_payload(encoded_json, endpoint_name):
    client = boto3.client("runtime.sagemaker")
    response = client.invoke_endpoint(
        EndpointName=endpoint_name, ContentType="application/json", Body=encoded_json
    )
    return response


query_response = query_endpoint_with_json_payload(
    json.dumps(payload).encode("utf-8"), endpoint_name=endpoint_name
)


def parse_response_multiple_texts(query_response):
    model_predictions = json.loads(query_response["Body"].read())
    generated_text = model_predictions["generated_texts"]
    return generated_text


generated_texts = parse_response_multiple_texts(query_response)
print(generated_texts)

['To make cheese fondue, first gather your ingredients. In a small bowl, combine the melted butter, sour cream, salt and pepper. Stir in the cheeses until melted. Transfer the fondue to ', 'Step 1: Preheat the oven to 350 degrees F (180 degrees C). Line a baking sheet with parchment paper. Step 2: In a large bowl, combine the melted butter, sour cream, salt, and pepper', 'To make cheese fondue, melt 2 sticks butter in a small saucepan over medium heat. Add 2 cups shredded Cheddar cheese and 1 cup chopped chives. Cook, stirring frequently, until the cheese is melted and', 'To make cheese fondue, combine 1 cup melted butter, 2 cups shredded cheddar cheese, 1 cup chopped chives, and 1 cup chopped parsley in a medium saucepan. Bring to a boil over medium heat,', 'Step 1: Preheat the oven to 375 degrees F (190 degrees C). In a large bowl, combine the melted butter and cheese. Stir to combine. Step 2: In a small bowl, combine the cornstarch']


In [ ]:
# Input must be a json
payload = {
    "text_inputs": "Tell me the steps to make pizza",
    "max_length": 50,
    "num_return_sequences": 3,
    "top_k": 50,
    "top_p": 0.95,
    "do_sample": True,
}


def query_endpoint_with_json_payload(encoded_json, endpoint_name):
    client = boto3.client("runtime.sagemaker")
    response = client.invoke_endpoint(
        EndpointName=endpoint_name, ContentType="application/json", Body=encoded_json
    )
    return response


query_response = query_endpoint_with_json_payload(
    json.dumps(payload).encode("utf-8"), endpoint_name=endpoint_name
)


def parse_response_multiple_texts(query_response):
    model_predictions = json.loads(query_response["Body"].read())
    generated_text = model_predictions["generated_texts"]
    return generated_text


generated_texts = parse_response_multiple_texts(query_response)
print(generated_texts)

In [26]:
text=fullDocs.iloc[6]['cleanText']

In [28]:
text = """Amazon Comprehend uses natural language processing (NLP) to extract insights about the content of documents. It develops insights by recognizing the entities, key phrases, language, sentiments, and other common elements in a document. Use Amazon Comprehend to create new products based on understanding the structure of documents. For example, using Amazon Comprehend you can search social networking feeds for mentions of products or scan an entire document repository for key phrases. 
You can access Amazon Comprehend document analysis capabilities using the Amazon Comprehend console or using the Amazon Comprehend APIs. You can run real-time analysis for small workloads or you can start asynchronous analysis jobs for large document sets. You can use the pre-trained models that Amazon Comprehend provides, or you can train your own custom models for classification and entity recognition. 
All of the Amazon Comprehend features accept UTF-8 text documents as the input. In addition, custom classification and custom entity recognition accept image files, PDF files, and Word files as input. 
Amazon Comprehend can examine and analyze documents in a variety of languages, depending on the specific feature. For more information, see Languages supported in Amazon Comprehend. Amazon Comprehend's Dominant language capability can examine documents and determine the dominant language for a far wider selection of languages."""


In [29]:
prompts = [
    "Briefly summarize this sentence: {text}",
    "Write a short summary for this text: {text}",
    "Generate a short summary this sentence:\n{text}",
    "{text}\n\nWrite a brief summary in a sentence or less",
    "{text}\nSummarize the aforementioned text in a single phrase.",
    "{text}\nCan you generate a short summary of the above paragraph?",
    "Write a sentence based on this summary: {text}",
    "Write a sentence based on '{text}'",
    "Summarize this article:\n\n{text}",
]

num_return_sequences = 3
parameters = {
    "max_length": 50,
    "num_return_sequences": num_return_sequences,
    "top_k": 50,
    "top_p": 0.95,
    "do_sample": True,
}

print(f"{bold}Number of return sequences are set as {num_return_sequences}{unbold}{newline}")
for each_prompt in prompts:
    payload = {"text_inputs": each_prompt.replace("{text}", text), **parameters}
    query_response = query_endpoint_with_json_payload(
        json.dumps(payload).encode("utf-8"), endpoint_name=endpoint_name
    )
    generated_texts = parse_response_multiple_texts(query_response)
    print(f"{bold} For prompt: '{each_prompt}'{unbold}{newline}")
    print(f"{bold} The {num_return_sequences} summarized results are{unbold}:{newline}")
    for idx, each_generated_text in enumerate(generated_texts):
        print(f"{bold}Result {idx}{unbold}: {each_generated_text}{newline}")

Number of return sequences are set as 3

 For prompt: 'Briefly summarize this sentence: {text}'

 The 3 summarized results are:

Result 0: Amazon Comprehend is a natural language processing (NLP) tool that analyzes text documents. Use Amazon Comprehend to create new products based on understanding the structure of documents. Access Amazon Comprehend using the

Result 1: Understand how Amazon Comprehend works. Use Amazon Comprehend to analyze documents.

Result 2: Understand how Amazon Comprehend works. Use Amazon Comprehend to analyze documents.

 For prompt: 'Write a short summary for this text: {text}'

 The 3 summarized results are:

Result 0: Amazon Comprehend helps you analyze documents to extract insights about their content.

Result 1: Amazon Comprehend uses natural language processing (NLP) to extract insights about the content of documents. Use Amazon Comprehend to create new products based on understanding the structure of documents. Access Amazon Comprehend document analysis

In [ ]:


premise = "The world cup has kicked off in Los Angeles, United States."
hypothesis = "The world cup takes place in United States."
options = """["yes", "no"]"""



In [ ]:
prompts = [
    """{premise}\n\nBased on the paragraph above can we conclude that "\"{hypothesis}\"?\n\n{options_}""",
    """{premise}\n\nBased on that paragraph can we conclude that this sentence is true?\n{hypothesis}\n\n{options_}""",
    """{premise}\n\nCan we draw the following conclusion?\n{hypothesis}\n\n{options_}""",
    """{premise}\nDoes this next sentence follow, given the preceding text?\n{hypothesis}\n\n{options_}""",
    """{premise}\nCan we infer the following?\n{hypothesis}\n\n{options_}""",
    """Read the following paragraph and determine if the hypothesis is true:\n\n{premise}\n\nHypothesis: {hypothesis}\n\n{options_}""",
    """Read the text and determine if the sentence is true:\n\n{premise}\n\nSentence: {hypothesis}\n\n{options_}""",
    """Can we draw the following hypothesis from the context? \n\nContext:\n\n{premise}\n\nHypothesis: {hypothesis}\n\n{options_}""",
    """Determine if the sentence is true based on the text below:\n{hypothesis}\n\n{premise}\n{options_}""",
]

parameters = {
    "max_length": 50,
    "num_return_sequences": 1,
    "top_k": 50,
    "top_p": 0.95,
    "do_sample": True,
}


for each_prompt in prompts:
    input_text = each_prompt.replace("{premise}", premise)
    input_text = input_text.replace("{hypothesis}", hypothesis)
    input_text = input_text.replace("{options_}", options)
    print(f"{bold} For prompt{unbold}: '{input_text}'{newline}")
    payload = {"text_inputs": input_text, **parameters}
    query_response = query_endpoint_with_json_payload(
        json.dumps(payload).encode("utf-8"), endpoint_name=endpoint_name
    )
    generated_texts = parse_response_multiple_texts(query_response)
    print(f"{bold} The reasoning result is{unbold}: '{generated_texts}'{newline}")

In [30]:


question = "Which plan is recommended for GPT-J?"



In [31]:
prompts = [
    """Context: NLP Cloud was founded in 2021 when the team realized there was no easy way to reliably leverage Natural Language Processing in production.
    Question: When was NLP Cloud founded?
    Answer: 2021
    ###
    Context: NLP Cloud developed their API by mid-2020 and they added many pre-trained open-source models since then.
    Question: What did NLP Cloud develop?
    Answer: API
    ###
    Context: All plans can be stopped anytime. You only pay for the time you used the service. In case of a downgrade, you will get a discount on your next invoice.
    Question: When can plans be stopped?
    Answer: Anytime
    ###
    Context: The main challenge with GPT-J is memory consumption. Using a GPU plan is recommended.
    Question: {question}
    Answer:"""
]


parameters = {
    "max_length": 350,
    "num_return_sequences": 1,
    "top_p": 0.1,
    "do_sample": True,
}


for each_prompt in prompts:
    input_text = each_prompt.replace("{question}", question)
    print(f"{bold} For prompt{unbold}: '{each_prompt}'{newline}")

    payload = {"text_inputs": input_text, **parameters}
    query_response = query_endpoint_with_json_payload(
        json.dumps(payload).encode("utf-8"), endpoint_name=endpoint_name
    )
    generated_texts = parse_response_multiple_texts(query_response)
    print(f"{bold} The reasoning result is{unbold}: '{generated_texts}'{newline}")


 For prompt: 'Context: NLP Cloud was founded in 2021 when the team realized there was no easy way to reliably leverage Natural Language Processing in production.
    Question: When was NLP Cloud founded?
    Answer: 2021
    ###
    Context: NLP Cloud developed their API by mid-2020 and they added many pre-trained open-source models since then.
    Question: What did NLP Cloud develop?
    Answer: API
    ###
    Context: All plans can be stopped anytime. You only pay for the time you used the service. In case of a downgrade, you will get a discount on your next invoice.
    Question: When can plans be stopped?
    Answer: Anytime
    ###
    Context: The main challenge with GPT-J is memory consumption. Using a GPU plan is recommended.
    Question: {question}
    Answer:'

 The reasoning result is: '['GPU']'



In [32]:
for each_prompt in prompts:
    input_text = each_prompt.replace("{question}", question)
    print(f"{bold} For prompt{unbold}: '{each_prompt}'{newline}")

    payload = {"text_inputs": input_text, **parameters}
    query_response = query_endpoint_with_json_payload(
        json.dumps(payload).encode("utf-8"), endpoint_name=endpoint_name
    )
    generated_texts = parse_response_multiple_texts(query_response)
    print(f"{bold} The reasoning result is{unbold}: '{generated_texts}'{newline}")


 For prompt: 'Context: NLP Cloud was founded in 2021 when the team realized there was no easy way to reliably leverage Natural Language Processing in production.
    Question: When was NLP Cloud founded?
    Answer: 2021
    ###
    Context: NLP Cloud developed their API by mid-2020 and they added many pre-trained open-source models since then.
    Question: What did NLP Cloud develop?
    Answer: API
    ###
    Context: All plans can be stopped anytime. You only pay for the time you used the service. In case of a downgrade, you will get a discount on your next invoice.
    Question: When can plans be stopped?
    Answer: Anytime
    ###
    Context: The main challenge with GPT-J is memory consumption. Using a GPU plan is recommended.
    Question: {question}
    Answer:'

 The reasoning result is: '['GPU']'



In [55]:
question='What is the recommended cheese for a Swiss cheese fondue'

In [58]:
prompts = [
    """Context: The Swiss cheese fondue is best made of 50% of Gruyere cheese .
    Question: What percentage of Gruyere is needed in the Swiss cheese fondue?
    Answer: 50%
    ###
    Context: The other 50% of a Swiss cheese fondue is Vacherin Fribourgeois.
    Question: What should be the other 50% in teh Swiss cheese fondue?
    Answer: Vacherin
    ###
    Context: The swiss cheese fondue with vacherin and gruyere is called moitié-moitié
    Question: How is the swiss chesse findue called ?
    Answer: Moitié-moitié
    ###
    Context: The recommended cheese for a Swiss cheese fondue moitié-moitié are Gruyere or Vacherin
    Question: {question}
    Answer:"""
]

In [59]:

parameters = {
    "max_length": 350,
    "num_return_sequences": 1,
    "top_p": 0.1,
    "do_sample": True,
}


for each_prompt in prompts:
    input_text = each_prompt.replace("{question}", question)
    print(f"{bold} For prompt{unbold}: '{each_prompt}'{newline}")

    payload = {"text_inputs": input_text, **parameters}
    query_response = query_endpoint_with_json_payload(
        json.dumps(payload).encode("utf-8"), endpoint_name=endpoint_name
    )
    generated_texts = parse_response_multiple_texts(query_response)
    print(f"{bold} The reasoning result is{unbold}: '{generated_texts}'{newline}")


 For prompt: 'Context: The Swiss cheese fondue is best made of 50% of Gruyere cheese .
    Question: What percentage of Gruyere is needed in the Swiss cheese fondue?
    Answer: 50%
    ###
    Context: The other 50% of a Swiss cheese fondue is Vacherin Fribourgeois.
    Question: What should be the other 50% in teh Swiss cheese fondue?
    Answer: Vacherin
    ###
    Context: The swiss cheese fondue with vacherin and gruyere is called moitié-moitié
    Question: How is the swiss chesse findue called ?
    Answer: Moitié-moitié
    ###
    Context: The recommended cheese for a Swiss cheese fondue moitié-moitié are Gruyere or Vacherin
    Question: {question}
    Answer:'

 The reasoning result is: '['Gruyere']'



In [65]:
question2='What is the other recommended cheese apart from Gruyere  for a Swiss cheese fondue'

In [68]:
prompts = [
    """Context: The Swiss cheese fondue is best made of 50% of Gruyere cheese .
    Question: What percentage of Gruyere is needed in the Swiss cheese fondue?
    Answer: 50%
    ###
    Context: The other 50% of a Swiss cheese fondue is Vacherin Fribourgeois.
    Question: What should be the other 50% in teh Swiss cheese fondue?
    Answer: Vacherin Fribourgeois
    ###
    Context: The swiss cheese fondue with vacherin and gruyere is called moitié-moitié
    Question: How is the swiss chesse findue called ?
    Answer: Moitié-moitié
    ###
    Context: The recommended cheese for a Swiss cheese fondue moitié-moitié are Gruyere or Vacherin Frbourgeois
    Question: {question2}
    Answer:"""
]

In [70]:

parameters = {
    "max_length": 350,
    "num_return_sequences": 1,
    "top_p": 0.1,
    "do_sample": True,
}


for each_prompt in prompts:
    input_text = each_prompt.replace("{question2}", question2)
    print(f"{bold} For prompt{unbold}: '{each_prompt}'{newline}")

    payload = {"text_inputs": input_text, **parameters}
    query_response = query_endpoint_with_json_payload(
        json.dumps(payload).encode("utf-8"), endpoint_name=endpoint_name
    )
    generated_texts = parse_response_multiple_texts(query_response)
    print(f"{bold} The reasoning result is{unbold}: '{generated_texts}'{newline}")


 For prompt: 'Context: The Swiss cheese fondue is best made of 50% of Gruyere cheese .
    Question: What percentage of Gruyere is needed in the Swiss cheese fondue?
    Answer: 50%
    ###
    Context: The other 50% of a Swiss cheese fondue is Vacherin Fribourgeois.
    Question: What should be the other 50% in teh Swiss cheese fondue?
    Answer: Vacherin Fribourgeois
    ###
    Context: The swiss cheese fondue with vacherin and gruyere is called moitié-moitié
    Question: How is the swiss chesse findue called ?
    Answer: Moitié-moitié
    ###
    Context: The recommended cheese for a Swiss cheese fondue moitié-moitié are Gruyere or Vacherin Frbourgeois
    Question: {question2}
    Answer:'

 The reasoning result is: '['Vacherin Frbourgeois']'

